In [ ]:
import time
import requests
import pandas as pd
from io import StringIO
from pathlib import Path

BASE_URL = "https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min"

AUTH_KEY = ""
STN = "368"

OUT_CSV = Path("./data/actual/AWS_368.csv")
N_CALLS = 50 # 총 1000번

WINDOW_HOURS = 20 # 12시간

STEP_MINUTES = 1 # 마지막 관측시각 + 1분

TIMEOUT_SEC = 60 # 60초 동안 기다림
SLEEP_SEC = 1 # 너무 빠르면 막힐 수 있어 약간 쉬기(필요 시 조절)

COLS = [
    "YYMMDDHHMI", "STN",
    "WD1", "WS1", "WDS", "WSS",
    "WD10", "WS10",
    "TA", "RE",
    "RN_15m", "RN_60m", "RN_12H", "RN_DAY",
    "HM", "PA", "PS", "TD"
]

In [6]:
def fetch_text(tm1: str, tm2: str) -> str:
    params = {
        "tm1": tm1,
        "tm2": tm2,
        "stn": STN,
        "disp": "1",
        "help": "0",
        "authKey": AUTH_KEY
    }
    r = requests.get(BASE_URL, params=params, timeout=TIMEOUT_SEC)
    r.raise_for_status()
    return r.text

In [8]:
def parse_kma_text_to_df(text: str) -> pd.DataFrame:
    # 1) 주석/빈줄 제거
    lines = []
    for ln in text.splitlines():
        ln = ln.strip()
        if not ln or ln.startswith("#"):
            continue
        lines.append(ln)

    if not lines:
        return pd.DataFrame(columns=COLS)

    # 2) 끝 토큰 '=' 제거
    cleaned = []
    for ln in lines:
        if ln.endswith(",="):
            ln = ln[:-2]
        elif ln.endswith("="):
            ln = ln[:-1]
            if ln.endswith(","):
                ln = ln[:-1]
        cleaned.append(ln)

    df = pd.read_csv(StringIO("\n".join(cleaned)), header=None, dtype=str)

    # 열수 체크
    if df.shape[1] != len(COLS):
        # 포맷이 바뀌었거나 에러 메시지가 섞인 경우가 흔함
        raise ValueError(f"컬럼 수 불일치: expected={len(COLS)}, got={df.shape[1]}")

    df.columns = COLS

    # datetime 파생
    df["datetime"] = pd.to_datetime(df["YYMMDDHHMI"], format="%Y%m%d%H%M", errors="coerce")

    # 숫자 변환
    for c in df.columns:
        if c in ("YYMMDDHHMI", "datetime"):
            continue
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # 결측 센티넬(-99.x) 처리
    num_cols = [c for c in df.columns if c not in ("YYMMDDHHMI", "datetime")]
    for c in num_cols:
        df.loc[df[c] <= -99, c] = pd.NA

    return df

In [9]:
def dt_to_tm(dt: pd.Timestamp) -> str:
    # KST 타임존을 별도로 붙이지 않아도, 문자열 기준으로 굴리면 충분함(네 데이터가 KST임)
    return dt.strftime("%Y%m%d%H%M")

In [10]:
def append_df_to_csv(df: pd.DataFrame, path: Path):
    if df.empty:
        return
    write_header = not path.exists()
    df.to_csv(path, mode="a", index=False, encoding="utf-8-sig", header=write_header)


In [11]:
def infer_next_tm1_from_existing_csv(path: Path, fallback_tm1: str) -> str:
    if not path.exists():
        return fallback_tm1

    # 큰 파일이면 tail 방식이 좋지만, 여기선 간단하게 마지막 행만 읽음
    last = pd.read_csv(path, usecols=["YYMMDDHHMI"]).tail(1)
    if last.empty:
        return fallback_tm1

    last_dt = pd.to_datetime(last["YYMMDDHHMI"].iloc[0], format="%Y%m%d%H%M", errors="coerce")
    if pd.isna(last_dt):
        return fallback_tm1

    next_dt = last_dt + pd.Timedelta(minutes=STEP_MINUTES)
    return dt_to_tm(next_dt)

In [ ]:
INITIAL_TM1 = "202601130120"  # 첫 실행용 기본값
tm1 = infer_next_tm1_from_existing_csv(OUT_CSV, INITIAL_TM1)

for i in range(N_CALLS):
    start_dt = pd.to_datetime(tm1, format="%Y%m%d%H%M")
    end_dt = start_dt + pd.Timedelta(hours=WINDOW_HOURS)
    tm2 = dt_to_tm(end_dt)

    print(f"[{i+1}/{N_CALLS}] tm1={tm1} tm2={tm2}")

    try:
        text = fetch_text(tm1, tm2)
        df = parse_kma_text_to_df(text)

        # 혹시 중복이 생기면(이어받기/겹침) 제거
        if not df.empty:
            df = df.drop_duplicates(subset=["YYMMDDHHMI", "STN"], keep="last")

        append_df_to_csv(df, OUT_CSV)

        # 다음 tm1 = 이번 응답의 마지막 datetime + 1분
        if df.empty or df["datetime"].isna().all():
            # 데이터가 비었으면 그냥 윈도우를 앞으로 한 칸 이동(12시간)
            tm1 = dt_to_tm(end_dt + pd.Timedelta(minutes=STEP_MINUTES))
        else:
            last_dt = df["datetime"].max()
            tm1 = dt_to_tm(last_dt + pd.Timedelta(minutes=STEP_MINUTES))

    except Exception as e:
        # 에러 나면 너무 멀리 점프하지 말고, 일단 12시간 뒤로만 이동해서 계속
        print("  [error]", repr(e))
        tm1 = dt_to_tm(end_dt + pd.Timedelta(minutes=STEP_MINUTES))

    time.sleep(SLEEP_SEC)

print("DONE:", OUT_CSV.resolve())

[1/50] tm1=202601130120 tm2=202601132120
[2/50] tm1=202601131109 tm2=202601140709
[3/50] tm1=202601131844 tm2=202601141444
[4/50] tm1=202601132224 tm2=202601141824
[5/50] tm1=202601141825 tm2=202601151425
[6/50] tm1=202601150325 tm2=202601152325
[7/50] tm1=202601152326 tm2=202601161926
[8/50] tm1=202601160826 tm2=202601170426
[9/50] tm1=202601161554 tm2=202601171154
[10/50] tm1=202601162344 tm2=202601171944
[11/50] tm1=202601170734 tm2=202601180334
[12/50] tm1=202601171809 tm2=202601181409
[13/50] tm1=202601180504 tm2=202601190104
[14/50] tm1=202601181621 tm2=202601191221
[15/50] tm1=202601190106 tm2=202601192106
[16/50] tm1=202601191053 tm2=202601200653
[17/50] tm1=202601192150 tm2=202601201750
[18/50] tm1=202601200731 tm2=202601210331
[19/50] tm1=202601200802 tm2=202601210402
[20/50] tm1=202601202051 tm2=202601211651
  [error] HTTPError('504 Server Error: Gateway Timeout for url: https://apihub.kma.go.kr/api/typ01/cgi-bin/url/nph-aws2_min?tm1=202601202051&tm2=202601211651&stn=368&dis

In [ ]:
import pandas as pd

# --- 설정 ---
FREQ_MIN = 1                 # AWS 1분 자료 가정
MAX_FETCH_HOURS = 20         # 한 번 호출할 최대 윈도우(너 코드와 맞춤)
SLEEP_SEC = 1                # 너무 빠른 호출 방지
RETRY = 3                    # 각 구간 재시도 횟수

In [ ]:
def load_existing_csv(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, dtype=str)
    if "datetime" not in df.columns:
        df["datetime"] = pd.to_datetime(df["YYMMDDHHMI"], format="%Y%m%d%H%M", errors="coerce")
    else:
        df["datetime"] = pd.to_datetime(df["datetime"], errors="coerce")
    df = df.dropna(subset=["datetime"]).sort_values("datetime")
    return df

In [ ]:
def find_missing_intervals(df: pd.DataFrame, freq_min: int = 1):
    """
    df(datetime 정렬됨)에서 누락 구간을 [(start_dt, end_dt), ...]로 반환
    start_dt/end_dt는 '누락된' 구간의 양끝(둘 다 포함)
    """
    dts = df["datetime"].drop_duplicates().sort_values().to_numpy()
    if len(dts) < 2:
        return []

    missing = []
    step = pd.Timedelta(minutes=freq_min)

    for prev, nxt in zip(dts[:-1], dts[1:]):
        prev = pd.Timestamp(prev)
        nxt = pd.Timestamp(nxt)
        if (nxt - prev) > step:
            start = prev + step
            end = nxt - step
            missing.append((start, end))

    # 인접/겹침 병합(안전)
    merged = []
    for s, e in missing:
        if not merged:
            merged.append([s, e])
        else:
            ps, pe = merged[-1]
            if s <= pe + step:
                merged[-1][1] = max(pe, e)
            else:
                merged.append([s, e])
    return [(a, b) for a, b in merged]

In [ ]:
def chunk_interval(start_dt: pd.Timestamp, end_dt: pd.Timestamp, max_hours: int, freq_min: int = 1):
    """
    inclusive API 기준으로 누락 구간(start_dt~end_dt)을 호출 가능한 크기로 쪼갬.
    각 chunk는 [tm1, tm2] inclusive.
    다음 chunk 시작은 tm2 + freq_min.
    반환: [(tm1, tm2), ...] (문자열 YYYYmmddHHMM)
    """
    chunks = []
    cur = start_dt
    max_span = pd.Timedelta(hours=max_hours)

    while cur <= end_dt:
        tm1 = dt_to_tm(cur)
        tm2_dt = min(cur + max_span, end_dt)
        # tm2는 inclusive로 받고 싶으면 +1분을 주는 방식도 있음.
        # 여기선 end_dt까지 덮게 하려면 tm2_dt를 그대로 쓰고,
        # API가 tm2를 포함해 주는지 애매하면 +1분 해도 됨.
        tm2 = dt_to_tm(tm2_dt)
        chunks.append((tm1, tm2))
        cur = tm2_dt + pd.Timedelta(minutes=freq_min)
    return chunks

In [ ]:
def refetch_missing_and_merge(csv_path: Path):
    if not csv_path.exists():
        raise FileNotFoundError(csv_path)

    base = load_existing_csv(csv_path)
    gaps = find_missing_intervals(base, freq_min=FREQ_MIN)

    print(f"Found gaps: {len(gaps)}")
    if not gaps:
        print("No missing intervals. Done.")
        return

    new_rows = []

    for gi, (gs, ge) in enumerate(gaps, start=1):
        print(f"\n[GAP {gi}/{len(gaps)}] missing {gs} ~ {ge} (KST)")
        chunks = chunk_interval(gs, ge, MAX_FETCH_HOURS, freq_min=FREQ_MIN)

        for ci, (tm1, tm2) in enumerate(chunks, start=1):
            print(f"  - chunk {ci}/{len(chunks)} tm1={tm1} tm2={tm2}")
            ok = False
            last_err = None

            for r in range(RETRY):
                try:
                    text = fetch_text(tm1, tm2)
                    df_new = parse_kma_text_to_df(text)
                    if not df_new.empty:
                        new_rows.append(df_new)
                    ok = True
                    break
                except Exception as e:
                    last_err = e
                    time.sleep(1.0)  # 재시도 간격

            if not ok:
                print("    [failed]", repr(last_err))

            time.sleep(SLEEP_SEC)

    if not new_rows:
        print("\nNo data fetched for gaps (all failed or empty).")
        return

    add = pd.concat(new_rows, ignore_index=True)

    # 기존 + 추가 병합
    merged = pd.concat([base, add], ignore_index=True)

    # 타입/정렬/중복 제거
    merged["datetime"] = pd.to_datetime(merged["YYMMDDHHMI"], format="%Y%m%d%H%M", errors="coerce")
    merged = merged.dropna(subset=["datetime"]).sort_values("datetime")

    # 동일 시각·동일 STN 중복 제거(최신값 유지)
    if "STN" not in merged.columns:
        merged["STN"] = str(STN)
    merged = merged.drop_duplicates(subset=["YYMMDDHHMI", "STN"], keep="last")

    # 최종 저장(덮어쓰기)
    merged.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print("\nDONE. Filled gaps and rewrote:", csv_path.resolve())

In [ ]:
# 실행
refetch_missing_and_merge(OUT_CSV)

Found gaps: 2

[GAP 1/2] missing 2026-01-13 21:21:00 ~ 2026-01-14 09:09:00 (KST)
  - chunk 1/1 tm1=202601132121 tm2=202601140909

[GAP 2/2] missing 2026-01-23 18:58:00 ~ 2026-01-24 08:04:00 (KST)
  - chunk 1/1 tm1=202601231858 tm2=202601240804

DONE. Filled gaps and rewrote: C:\project_WWTP\python\data\actual\AWS_368.csv
